In [5]:
# Import basic libraries... you might need to type 'pip install <library>' in the terminal to install them
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Import the libraries for the machine learning model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder # converting target variable
from sklearn.preprocessing import OrdinalEncoder # converting categorical feature variables
from sklearn.naive_bayes import CategoricalNB, GaussianNB # creates the model
from sklearn.metrics import accuracy_score # to compute accuracy score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay # confusion matrix as a heatmap
from sklearn.model_selection import cross_val_score # for cross-validation

In [11]:
# display all columns in the dataframe
pd.options.display.max_columns = None

### Loading the dataset

In [3]:
# Load churn dataset
df = pd.read_csv('churn.csv')

print(df.head)

<bound method NDFrame.head of        age gender security_no region_category  membership_category  \
0       18      F     XW0DQ7H         Village  Platinum Membership   
1       32      F     5K0N3X1            City   Premium Membership   
2       44      F     1F2TCL3            Town        No Membership   
3       37      M     VJGJ33N            City        No Membership   
4       31      F     SVZXCWB            City        No Membership   
...    ...    ...         ...             ...                  ...   
36987   46      F     6F51HFO             NaN     Basic Membership   
36988   29      F     21KSM8Y            Town     Basic Membership   
36989   23      F     XK1IM9H             NaN     Basic Membership   
36990   53      M     K6VTP1Z         Village  Platinum Membership   
36991   35      M     LBX0GLR            Town    Silver Membership   

      joining_date joined_through_referral referral_id  \
0       17-08-2017                      No    xxxxxxxx   
1       28-08

### Data Cleaning / Munging / Processing
- formatting issues
- dealing with missing data
- filtering to subset of interest
- handling missing data (do we drop them or imputate (give them another value))

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36992 entries, 0 to 36991
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   age                           36992 non-null  int64  
 1   gender                        36992 non-null  object 
 2   security_no                   36992 non-null  object 
 3   region_category               31564 non-null  object 
 4   membership_category           36992 non-null  object 
 5   joining_date                  36992 non-null  object 
 6   joined_through_referral       36992 non-null  object 
 7   referral_id                   36992 non-null  object 
 8   preferred_offer_types         36704 non-null  object 
 9   medium_of_operation           36992 non-null  object 
 10  internet_option               36992 non-null  object 
 11  last_visit_time               36992 non-null  object 
 12  days_since_last_login         36992 non-null  int64  
 13  a

In [4]:
# check for missing values
print(df.isnull().sum())

age                                0
gender                             0
security_no                        0
region_category                 5428
membership_category                0
joining_date                       0
joined_through_referral            0
referral_id                        0
preferred_offer_types            288
medium_of_operation                0
internet_option                    0
last_visit_time                    0
days_since_last_login              0
avg_time_spent                     0
avg_transaction_value              0
avg_frequency_login_days           0
points_in_wallet                3443
used_special_discount              0
offer_application_preference       0
past_complaint                     0
complaint_status                   0
feedback                           0
churn_risk_score                   0
dtype: int64


In [12]:
# joining date format is inconsistent e.g. 1/1/2020, 01/01/2020, 1-1-2020
df_clean = df.copy() # make a copy of the original dataframe

# standardize the date format to datetime format
df_clean['joining_date'] = pd.to_datetime(df_clean['joining_date'], errors='coerce', dayfirst=True)
df_clean.head(10)

,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,18,F,XW0DQ7H,Village,Platinum Membership,2017-08-17,No,xxxxxxxx,Gift Vouchers/Coupons,?,Wi-Fi,16:08:02,17,300.63,53005.25,17,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,0
1,32,F,5K0N3X1,City,Premium Membership,2017-08-28,?,CID21329,Gift Vouchers/Coupons,Desktop,Mobile_Data,12:38:13,16,306.34,12838.38,10,NaN,Yes,No,Yes,Solved,Quality Customer Care,0
2,44,F,1F2TCL3,Town,No Membership,NaT,Yes,CID12313,Gift Vouchers/Coupons,Desktop,Wi-Fi,22:53:21,14,516.16,21027.00,22,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,1
3,37,M,VJGJ33N,City,No Membership,2016-10-29,Yes,CID3793,Gift Vouchers/Coupons,Desktop,Mobile_Data,15:57:50,11,53.27,25239.56,6,567.66,No,Yes,Yes,Unsolved,Poor Website,1
4,31,F,SVZXCWB,City,No Membership,NaT,No,xxxxxxxx,Credit/Debit Card Offers,Smartphone,Mobile_Data,15:46:44,20,113.13,24483.66,16,663.06,No,Yes,Yes,Solved,Poor Website,1
5,13,M,PSG1LGF,City,Gold Membership,NaT,No,xxxxxxxx,Gift Vouchers/Coupons,?,Wi-Fi,6:46:07,23,433.62,13884.77,24,722.27,Yes,No,Yes,Unsolved,No reason specified,0
6,21,M,R3CX1EA,Town,Gold Membership,2015-03-19,Yes,CID24708,Gift Vouchers/Coupons,Desktop,Mobile_Data,11:40:04,10,55.38,8982.50,28,756.21,Yes,No,Yes,Solved in Follow-up,No reason specified,0
7,42,M,4UJ1551,NaN,No Membership,NaT,?,CID56614,Credit/Debit Card Offers,Both,Fiber_Optic,7:52:43,19,429.11,44554.82,24,568.08,No,Yes,Yes,Unsolved,Poor Product Quality,1
8,44,M,0481QNQ,Village,Silver Membership,2016-12-14,No,xxxxxxxx,Without Offers,Smartphone,Fiber_Optic,6:50:10,15,191.07,18362.31,20,NaN,Yes,No,Yes,Solved in Follow-up,Poor Customer Service,0
9,45,F,ZHP4MCR,Town,No Membership,2016-11-30,No,xxxxxxxx,Gift Vouchers/Coupons,?,Wi-Fi,19:10:16,10,97.31,19244.16,28,706.23,No,Yes,Yes,No Information Available,Poor Customer Service,1


In [20]:
# some variables have place holder strings (e.g. '?', 'xxxxxxxx', 'Error', 'None', 'N/A')
placeholders = ['?', 'xxxxxxx', 'Error', 'None']
df_clean.replace(placeholders, pd.NA, inplace=True)
df_clean.head(10)

,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,18,F,XW0DQ7H,Village,Platinum Membership,2017-08-17,No,xxxxxxxx,Gift Vouchers/Coupons,<NA>,Wi-Fi,16:08:02,17,300.63,53005.25,17.0,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,0
1,32,F,5K0N3X1,City,Premium Membership,2017-08-28,<NA>,CID21329,Gift Vouchers/Coupons,Desktop,Mobile_Data,12:38:13,16,306.34,12838.38,10.0,NaN,Yes,No,Yes,Solved,Quality Customer Care,0
2,44,F,1F2TCL3,Town,No Membership,NaT,Yes,CID12313,Gift Vouchers/Coupons,Desktop,Wi-Fi,22:53:21,14,516.16,21027.00,22.0,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,1
3,37,M,VJGJ33N,City,No Membership,2016-10-29,Yes,CID3793,Gift Vouchers/Coupons,Desktop,Mobile_Data,15:57:50,11,53.27,25239.56,6.0,567.66,No,Yes,Yes,Unsolved,Poor Website,1
4,31,F,SVZXCWB,City,No Membership,NaT,No,xxxxxxxx,Credit/Debit Card Offers,Smartphone,Mobile_Data,15:46:44,20,113.13,24483.66,16.0,663.06,No,Yes,Yes,Solved,Poor Website,1
5,13,M,PSG1LGF,City,Gold Membership,NaT,No,xxxxxxxx,Gift Vouchers/Coupons,<NA>,Wi-Fi,6:46:07,23,433.62,13884.77,24.0,722.27,Yes,No,Yes,Unsolved,No reason specified,0
6,21,M,R3CX1EA,Town,Gold Membership,2015-03-19,Yes,CID24708,Gift Vouchers/Coupons,Desktop,Mobile_Data,11:40:04,10,55.38,8982.50,28.0,756.21,Yes,No,Yes,Solved in Follow-up,No reason specified,0
7,42,M,4UJ1551,NaN,No Membership,NaT,<NA>,CID56614,Credit/Debit Card Offers,Both,Fiber_Optic,7:52:43,19,429.11,44554.82,24.0,568.08,No,Yes,Yes,Unsolved,Poor Product Quality,1
8,44,M,0481QNQ,Village,Silver Membership,2016-12-14,No,xxxxxxxx,Without Offers,Smartphone,Fiber_Optic,6:50:10,15,191.07,18362.31,20.0,NaN,Yes,No,Yes,Solved in Follow-up,Poor Customer Service,0
9,45,F,ZHP4MCR,Town,No Membership,2016-11-30,No,xxxxxxxx,Gift Vouchers/Coupons,<NA>,Wi-Fi,19:10:16,10,97.31,19244.16,28.0,706.23,No,Yes,Yes,No Information Available,Poor Customer Service,1


In [21]:
# convert 'avg_frequency_login_days' to numeric
df_clean['avg_frequency_login_days'] = pd.to_numeric(df_clean['avg_frequency_login_days'], errors='coerce')

In [22]:
# check missing values after cleaning
missing_summary = df_clean.isna().sum().sort_values(ascending=False)
missing_summary

joining_date                    14665
joined_through_referral          5438
region_category                  5428
medium_of_operation              5393
avg_frequency_login_days         3522
points_in_wallet                 3443
preferred_offer_types             288
age                                 0
avg_transaction_value               0
feedback                            0
complaint_status                    0
past_complaint                      0
offer_application_preference        0
used_special_discount               0
last_visit_time                     0
avg_time_spent                      0
days_since_last_login               0
gender                              0
internet_option                     0
referral_id                         0
membership_category                 0
security_no                         0
churn_risk_score                    0
dtype: int64

In [18]:
# check data types after cleaning
print(df_clean.dtypes)

age                                      int64
gender                                  object
security_no                             object
region_category                         object
membership_category                     object
joining_date                    datetime64[ns]
joined_through_referral                 object
referral_id                             object
preferred_offer_types                   object
medium_of_operation                     object
internet_option                         object
last_visit_time                         object
days_since_last_login                    int64
avg_time_spent                         float64
avg_transaction_value                  float64
avg_frequency_login_days               float64
points_in_wallet                       float64
used_special_discount                   object
offer_application_preference            object
past_complaint                          object
complaint_status                        object
feedback     

### Exploratory Data Analysis
- gain insights here
- identify patterns and draw interesting conclusions
- clarify all variables of interest (maybe we can do feature selection of some kind)
    - features vs. target variables
    - descriptive statistics
    - distributions
- explore relationships between variables
    - scatterplots, simple regressions
    - correlations
- Disaggregate
    - over space (city, latitude, etc.)
    - over time (or time period)
    - by classification (customers that churn vs. did not, age group, male vs. female, etc.)
    - by other segments
